# Water Scarcity and Global Conflict Analysis
This project aims to explore the complex relationship between armed conflict and water scarcity by integrating and analyzing datasets from various sources. We will leverage geospatial and environmental data to assess how water scarcity influences the occurrence and intensity of conflicts.

## Definitions
- Scarcity: Demand for a good or service is greater than the availability of the good or service (Oxford Languages).
- Supply: total freshwater resources available in cubic meters per person, per year (The ImpEE Project).
- Withdrawal: amount extracted for use by country (The ImpEE Project).
- Water Stress: ratio between total freshwater withdrawn (TFWW) and total renewable freshwater resources (TRWR). Water stress = TFWW / TRWR (Wikipedia).
- Water Scarcity: volume of fresh water available does not meet the per person per day recommendations for human health (University of Nottingham).er day

In [8]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from functools import partial, reduce

## Data Collection and Cleanup

#### Collect csv files

In [9]:
# Collect water scarcity data from the Food and Agriculture Organization (FAO) https://data.apps.fao.org/aquastat/?lang=en
aqua_csv = pd.read_csv('Resources/AQUASTAT Dissemination System.csv')

# Collect international conflict data from the University of Alabama https://internationalconflict.ua.edu/data-download/
mie_csv = pd.read_csv('Resources/ua-mie-1.0.csv')
micnames = pd.read_csv('Resources/ua-micnames-1.0.csv')

# Collect country codes from Correlates of War (COW) https://correlatesofwar.org/data-sets/cow-country-codes-2/
COW_Country_Codes = pd.read_csv('Resources/COW-country-codes.csv')

#### Cleanup the Militarized Interstate Events (MIE) csv file

In [11]:
# Copy the dataframe with only the columns we want 
mie_df = mie_csv[['styear', 'ccode1', 'eventnum', 'micnum', 'hostlev', 'ccode2']].copy()

# Create a dictionary for the country codes and their names and the confrontation codes and their name
code_to_country = pd.Series(COW_Country_Codes.StateNme.values, index=COW_Country_Codes.CCode).to_dict()
conflict_name = pd.Series(micnames.micname.values, index= micnames.micnum).to_dict()

# Map the country codes to their names from the dictionary and replace
mie_df['ccode1'] = mie_df['ccode1'].map(code_to_country)
mie_df['ccode2'] = mie_df['ccode2'].map(code_to_country)
mie_df['micnum'] = mie_df['micnum'].map(conflict_name)

# Rename columns headers
mie_df_clean = mie_df.rename(columns={'styear': 'Year',
                                'ccode1': 'Country',
                                'ccode2': 'Target Country',
                                'eventnum': 'Event Number',
                                'micnum': 'Conflict Name',
                                'hostlev': 'Hosility Level'
                                })

#### Cleanup the AQUASTAT csv file

In [12]:
# Copy the dataframe with only the columns we want 
aqua_df = aqua_csv[['Year', 'Area', 'Variable', 'Value', 'Unit']].copy()

# Rename column header
aqua_df = aqua_df.rename(columns={'Area': 'Country'})

# Replace country with dictionary values
aqua_df['Country'] = aqua_df['Country'].replace(code_to_country)

# Copy the dataframe with only the columns we want 
aqua_df = aqua_csv[['Year', 'Area', 'Variable', 'Value', 'Unit']].copy()

# Rename column header
aqua_df = aqua_df.rename(columns={'Area': 'Country'})
aqua_df['Country'] = aqua_df['Country'].replace(code_to_country)

In [13]:
# Create columns for all of the variables in the export and create a dataframe with all ths

# Create dataframe for Human Capital Index (max value = 1)
hdi_df = aqua_df.loc[aqua_df['Variable'] == 'Human Development Index (HDI)']
hdi_df = hdi_df.rename(columns={'Value': 'HDI'}).drop(columns=['Variable', 'Unit'])

# Create dataframe for Pop Density (ppl/km2)
pop_dens_df = aqua_df.loc[aqua_df['Variable'] == 'Population density']
pop_dens_df = pop_dens_df.rename(columns={'Value': 'Pop Density'}).drop(columns=['Variable', 'Unit'])

# Create dataframe for Wtr Stress %
wstress_df = aqua_df.loc[aqua_df['Variable'] == 'SDG 6.4.2. Water Stress']
wstress_df = wstress_df.rename(columns={'Value': 'Wtr Stress'}).drop(columns=['Variable', 'Unit'])

# Create dataframe for Total exploitable water resources (1b m3/yr)
tw_res_df = aqua_df.loc[aqua_df['Variable'] == 'Total exploitable water resources']
tw_res_df = tw_res_df.rename(columns={'Value': 'Tot Wtr Resource'}).drop(columns=['Variable', 'Unit'])

# Create dataframe for Total freshwater withdrawal 1b m3/yr)
tfw_wdrl_df = aqua_df.loc[aqua_df['Variable'] == 'Total freshwater withdrawal']
tfw_wdrl_df = tfw_wdrl_df.rename(columns={'Value': 'FreshW Wdrl'}).drop(columns=['Variable', 'Unit'])

# Create dataframe for Total Population (1000ppl)
tpop_df = aqua_df.loc[aqua_df['Variable'] == 'Total population']
tpop_df = tpop_df.rename(columns={'Value': 'Total Population'}).drop(columns=['Variable', 'Unit'])

# Create dataframe for Total Water Withdrawl (ppl/km2)
twdrl_df = aqua_df.loc[aqua_df['Variable'] == 'Total water withdrawal']
twdrl_df = twdrl_df.rename(columns={'Value': 'Total Withdrawl'}).drop(columns=['Variable', 'Unit'])

# Create dataframe for Total water withdrawal per capita (m3/ppl/yr)
tw_wdrl_pc_df = aqua_df.loc[aqua_df['Variable'] == 'Total water withdrawal per capita']
tw_wdrl_pc_df = tw_wdrl_pc_df.rename(columns={'Value': 'Wtr Withdrawl'}).drop(columns=['Variable', 'Unit'])

# Make a list of dataframes
aqua_df_lst = [hdi_df, pop_dens_df, wstress_df, tw_res_df, tfw_wdrl_df, tpop_df, twdrl_df, tw_wdrl_pc_df]

# Create a clean dataframe for water data 
aqua_df_clean = reduce(lambda  left,right: pd.merge(left,right,on=['Year', 'Country'],how='outer'), aqua_df_lst)

In [16]:
# Merge the dataframes into a master dataframe for analysis
water_conflicts_df = pd.merge(aqua_df_clean, mie_df_clean, how='outer', on=['Year', 'Country'])

# Fill the NaN under conflicts with no conflict
water_conflicts_df['Conflict Name'] = water_conflicts_df['Conflict Name'].fillna('No Conflict')
water_conflicts_df = water_conflicts_df.fillna(0)

# Display the final merged clean dataframe for analysis
water_conflicts_df.tail()

,Year,Country,HDI,Pop Density,Wtr Stress,Tot Wtr Resource,FreshW Wdrl,Total Population,Total Withdrawl,Wtr Withdrawl,Event Number,Conflict Name,Hosility Level,Target Country
47356,2021,Western Asia,0.0,0.000000,62.920000,0.0,169.223560,289733.124,180.411929,622.683130,0.0,No Conflict,0.0,0
47357,2021,World,0.0,0.000000,18.550000,0.0,3949.091520,7915610.122,3990.183502,504.090454,0.0,No Conflict,0.0,0
47358,2021,Yemen,0.0,62.468779,169.761905,0.0,3.565000,32981.641,3.565000,108.090437,0.0,No Conflict,0.0,0
47359,2021,Zambia,0.0,25.874125,2.835498,0.0,1.572000,19473.125,1.572000,80.726642,0.0,No Conflict,0.0,0
47360,2021,Zimbabwe,0.0,40.929276,46.091620,1.5,4.909679,15993.524,4.909679,306.979212,0.0,No Conflict,0.0,0


In [7]:
# Write the new merged dataframe to a csv file
water_conflicts_df.to_csv('Resources/water_conflicts_df.csv')